In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

--2024-11-15 13:47:48--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144474 (141K) [text/plain]
Saving to: ‘train_split.csv’

train_split.csv     100%[===================>] 141.09K  --.-KB/s    in 0.005s  

2024-11-15 13:47:49 (26.4 MB/s) - ‘train_split.csv’ saved [144474/144474]

--2024-11-15 13:47:49--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35259 (34K) 

In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import pandas as pd
import shutil
import sys
from transformers import BertConfig, BertForSequenceClassification
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
import numpy as np
import torch.nn.functional as F

In [ ]:
data = pd.read_csv('train_split.csv')
data.head()

,text,Joy,Fear,Anger,Sadness,Surprise
0,The light would come all the way up to your ve...,0,1,0,0,1
1,"Well, my birthday is in January.",0,0,0,0,0
2,"Tears in my eyes, too much on my mind, and dee...",0,1,0,1,0
3,"My eyes scanned quickly into every room, every...",0,1,1,0,0
4,I felt the afterglow of the late afternoon sun...,1,0,0,0,0


In [ ]:
train_df = data['text']
train_df.head()

,text
0,The light would come all the way up to your ve...
1,"Well, my birthday is in January."
2,"Tears in my eyes, too much on my mind, and dee..."
3,"My eyes scanned quickly into every room, every..."
4,I felt the afterglow of the late afternoon sun...


In [ ]:
target_df = data[data.columns[1:]]
target_df.head()

,Joy,Fear,Anger,Sadness,Surprise
0,0,1,0,0,1
1,0,0,0,0,0
2,0,1,0,1,0
3,0,1,1,0,0
4,1,0,0,0,0


In [ ]:
#hyperparameters

max_len = 128
train_batch_size = 8
valid_batch_size = 8
batch_size = 16
epochs = 5
LEARNING_RATE = 1e-05

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, dataFrame, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataFrame
        self.text = dataFrame['text']
        self.max_len = max_len
        self.targets = self.data[dataFrame.columns[1:]]

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'targets': torch.FloatTensor(self.targets.iloc[index].values)
        }


In [ ]:
val_df = pd.read_csv('test_split.csv')

In [ ]:
data.shape

(1600, 6)

In [ ]:
val_df.shape

(400, 6)

In [ ]:
train_dataset = CustomDataset(data, tokenizer, max_len)
valid_dataset = CustomDataset(val_df, tokenizer, max_len)

In [ ]:
len(valid_dataset)

400

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = train_batch_size, num_workers = 4, pin_memory=True, shuffle = True)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = valid_batch_size, num_workers = 4, pin_memory=True, shuffle = False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    '''torch.save(state, f_path)
    # if it is best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)'''

    torch.save(model.state_dict(), f_path)
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.2)
        self.linear = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attn_mask, token_type_ids=None):  # RoBERTa does not need token_type_ids
        output = self.bert_model(input_ids, attention_mask=attn_mask)
        cls_output = output.last_hidden_state[:, 0, :]  # Use the [CLS] token
        output_dropout = self.dropout(cls_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.AdamW(params = model.parameters(), lr = LEARNING_RATE)


In [ ]:
val_target = []
val_output = []

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, optimizer, checkpoint_path, best_model_path):
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf

    for epoch in range(1, n_epochs + 1):
        train_loss = 0
        valid_loss = 0
        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            #print('yyy epoch',batch_idx)
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            #loss = loss_fn(outputs, targets)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))

        print('############# Epoch {}: Training End   #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))

    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_target.extend(targets.cpu().detach().numpy().tolist())
            val_output.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            #print('zzz epoch',batch_idx)

            # print training/validation statistics
        # calculate average loss over an epoch
        train_loss = train_loss / len(training_loader)
        valid_loss = valid_loss / len(validation_loader)
        print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
        ))

        # save checkpoints variables and add important data
        checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        # save checkpoint
        save_ckp(checkpoint, False, checkpoint_path, best_model_path)

        # if validation loss decreased, save the model
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
            # save checkpoint as best model
            save_ckp(checkpoint, True, checkpoint_path, best_model_path)
            valid_loss_min = valid_loss

    print('############# Epoch {}: Validation End   #############'.format(epoch))

    return model


In [ ]:
ckpt_path = '/content/drive/MyDrive/curr_ckpt'
best_model_path = '/content/drive/MyDrive/best_model.pt'

In [ ]:
#trained_model = train_model(epochs, train_data_loader, valid_data_loader, model, optimizer, ckpt_path, best_model_path)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import classification_report

In [ ]:
!git clone https://github.com/ashishkumar0803/21058_Ashish_Kumar_nlpassignment3.git

Cloning into '21058_Ashish_Kumar_nlpassignment3'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

def validation():
    model.eval()
    val_target = []
    val_output = []
    with torch.no_grad():
        for batch_idx, data in enumerate(valid_data_loader, 0):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)

            # Extend target and output lists
            val_target.extend(targets.cpu().detach().numpy().tolist())
            val_output.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    # Convert probabilities to binary predictions
    val_target = np.array(val_target)
    val_output = (np.array(val_output) >= 0.5).astype(int)

    # Generate classification report
    print(
        classification_report(
            val_target, val_output,
            target_names=['Joy', 'Sadness', 'Fear', 'Anger', 'Surprise'],
            zero_division=0
        )
    )

# Adjust DataLoader
valid_data_loader = DataLoader(valid_dataset, batch_size=valid_batch_size, num_workers=2, shuffle=False)

model.load_state_dict(torch.load('/content/21058_Ashish_Kumar_nlpassignment3/best_model_44.pt', map_location=device))

# Validate the model
validation()


<ipython-input-27-689f65439821>:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/21058_Ashish_Kumar_nlpassignment3/best_model_44.

              precision    recall  f1-score   support

         Joy       0.71      0.63      0.67        94
     Sadness       0.86      0.82      0.84       232
        Fear       0.80      0.23      0.36        52
       Anger       0.74      0.67      0.70       126
    Surprise       0.71      0.70      0.71       124

   micro avg       0.78      0.69      0.73       628
   macro avg       0.76      0.61      0.65       628
weighted avg       0.78      0.69      0.72       628
 samples avg       0.67      0.63      0.63       628

